In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math as m
import sys
import os 
import csv

In [3]:
movie_ratings = np.loadtxt("/Users/Fairy/Desktop/hw8/hw8_ratings_fa18.txt")
z_i=np.loadtxt("/Users/Fairy/Desktop/hw8/hw8_probZ_init.txt")
r_given_z=np.loadtxt("/Users/Fairy/Desktop/hw8/hw8_probRgivenZ_init.txt")

location=os.getcwd()

location+="\\Desktop\\HW8\\hw8_movieTitles_fa18.txt"

movies = pd.DataFrame.from_csv(location, sep = '\t', header = -1)
movies.reset_index(inplace=True)
movies.columns=['movie_titles']


C:\Users\Fairy\AnacodaContainer\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':


In [4]:
sanity=np.sum(np.array(movie_ratings) > 0, axis=0)/np.sum(np.array(movie_ratings) > -1, axis=0)

In [5]:
movies['popularity']=sanity

In [6]:
movies.sort_values('popularity',ascending=False)

,movie_titles,popularity
0,Inception,0.973094
1,The_Social_Network,0.936170
21,The_Dark_Knight_Rises,0.925581
39,Interstellar,0.917874
3,Shutter_Island,0.891473
28,Wolf_of_Wall_Street,0.881356
23,Django_Unchained,0.868421
2,Black_Swan,0.863636
51,Ready_Player_One,0.862903
33,12_Years_a_Slave,0.860465


In [7]:
iterations=129
k=4
print(len(z_i))
r_given_z[0]
movie_ratings[1]
len(movies)

4


62

In [8]:
def probability_rj(t):
    sum=0
    for i in range(len(z_i)):
        sum+=z_i[i]*prob_rj_given_z_t(t,i)
    return sum

In [9]:
def prob_rj_given_z_t(t,i):
    prod=1
    omegat=movie_ratings[t]
    for index,value in enumerate(omegat):
        #print(omegat)
        if(value==1):
            prod*=r_given_z[index][i]
        elif(value==0):
            prod*=1-r_given_z[index][i]
    return prod  

In [10]:
def e_step_prob_z_i_given_rj_t(i,t):
    roit=0
    numerator=z_i[i]*prob_rj_given_z_t(t,i)
    #print(numerator)
    sum=0
    for i_dash in range(len(z_i)):
        sum+=z_i[i_dash]*prob_rj_given_z_t(t,i_dash)
    roit=numerator/sum
    #print("rp",roit)
    return roit

In [25]:
def mstep(i,j,T):
    s=u=sum=0
    for t in range(T):
        sum+=e_step_prob_z_i_given_rj_t(i,t)
        #print(e_step_prob_z_i_given_rj_t(i,t))
        if(movie_ratings[t][j]==-1):
            s+=e_step_prob_z_i_given_rj_t(i,t)*r_given_z[j][i]
        elif(movie_ratings[t][j]==1):
            u+=e_step_prob_z_i_given_rj_t(i,t)
    #print("s",s)
    return (s+u)/sum,sum/T
                    

In [ ]:
T=len(movie_ratings)
ll=0
master={}
for t in range(T):
    ll+=np.log(probability_rj(t))
output={"iteration":0,"ll":ll/T}
master[0]=output
print(master)
for iter in range(1,iterations):
    x=np.zeros((len(movies),4))
    y=np.zeros(4)
    for i in range(4):
        for j in range(len(movies)):
            two,one=mstep(i,j,T)
            
            x[j][i]=two
            y[i]=one
    r_given_z=x
    z_i=y
    ll=0
    for t in range(T):
        ll+=np.log(probability_rj(t))
    output={"iteration":iter,"ll":ll/T}
    master[iter]=output
    print(output)

{0: {'iteration': 0, 'll': -26.678832965400435}}
{'iteration': 1, 'll': -16.094668997711192}
{'iteration': 2, 'll': -14.28779402734125}


In [43]:
likely_movies={}
person=91
for mov in range(len(movies)):
    d=0
    if(movie_ratings[person][mov]==-1):
        for i in range(k):
            d+=e_step_prob_z_i_given_rj_t(i,person)*r_given_z[mov][i]
        likely_movies[movies['movie_titles'][mov]]={"expected_val":d,"movie":mov+1}
        likely_movies[movies['movie_titles'][mov]]=d

In [44]:
likely_movies
output = pd.DataFrame()
output = output.append(likely_movies, ignore_index=True)

In [45]:
O=output.T
O.columns=['expected_values']
O.sort_values('expected_values',ascending=False)

,expected_values
Ready_Player_One,0.999475
Shutter_Island,0.890891
Django_Unchained,0.889025
Les_Miserables,0.888749
Manchester_by_the_Sea,0.857623
Black_Swan,0.850455
Room,0.839086
Darkest_Hour,0.837219
Three_Billboards_Outside_Ebbing,0.832105
Hidden_Figures,0.832045
